<a href="https://colab.research.google.com/github/AshwinRamachandran2002/soc/blob/main/Self_Drive_simulation/SelfDriveSumulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, UpSampling2D, Input, Dense, Permute, Reshape, Activation, Flatten
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D, Concatenate
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import regularizers
from keras.utils.vis_utils import plot_model as plot

def SelfDrivingCarModel(inputShape=(256, 256, 3)):
    dropoutAmount = 0.5
    poolSize = (2,2)
    
    inputLayer = Input(shape=inputShape)

    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputLayer)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=poolSize)(conv1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=poolSize)(conv2)

    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=poolSize)(conv3)

    conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(dropoutAmount)(conv4)
    pool4 = MaxPooling2D(pool_size=poolSize)(conv4)

    flat = Flatten()(pool4)
    
    steeringOutput = regressionHead(flat)
    throttleOutput = regressionHead(flat)
                        
    model = Model(inputs=inputLayer, outputs=[steeringOutput,throttleOutput])

    return model

def regressionHead(layer, dropoutAmount=0.5):
    dense1 = Dense(256, activation = "relu")(layer)
    drop6 = Dropout(dropoutAmount)(dense1)

    dense2 = Dense(128, activation = "relu")(drop6)
    drop7 = Dropout(dropoutAmount)(dense2)

    dense3 = Dense(64, activation = "relu")(drop7)
    drop8 = Dropout(dropoutAmount)(dense3)

    dense4 = Dense(24, activation = "relu")(drop8)
    drop9 = Dropout(dropoutAmount)(dense4)
    
    output = Dense(1, activation = "linear")(drop9)

    return output

In [2]:
import numpy as np
import os, cv2, time
from keras.models import load_model

NEW_SIZE = (400, 220)
#Model Variables
MODEL_NAME = "Self Driving Car.model"
model = SelfDrivingCarModel(inputShape=(NEW_SIZE[1], NEW_SIZE[0], 3))
model.load_weights('/content/drive/MyDrive/soc/Self Driving Car.model')
for layer in model.layers[:18]:
    layer.trainable = False
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 220, 400, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 220, 400, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 220, 400, 32) 9248        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 110, 200, 32) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [3]:
model.compile(optimizer=Adam(lr=2e-6),loss='categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 220, 400, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 220, 400, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 220, 400, 32) 9248        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 110, 200, 32) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
model.save('simulation.h5')

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/soc/file4.csv")
data.head()

,Unnamed: 0,image,steer
0,0.0,[[0.51764706 0.4745098 0.43921569 ... 0.33333...,NaN
1,NaN,image,steer
2,0.0,[[0.51764706 0.4745098 0.43529412 ... 0.33333...,0.0
3,NaN,image,steer
4,0.0,[[0.51372549 0.47058824 0.43529412 ... 0.32941...,0.0


In [ ]:
X_data_new= data['image']
Y_data=data['steer']



In [ ]:
history = model.fit(X_data_new, Y_data,validation_split=0.13,epochs=10,
                        batch_size=32)